In [3]:
pip install transformers[torch] accelerate==0.20.1

  Obtaining dependency information for accelerate==0.20.1 from https://files.pythonhosted.org/packages/77/a8/af1e480814d0cccfe60def63471841a365dbd6f94e2d308d9bc1e3db2da2/accelerate-0.20.1-py3-none-any.whl.metadata
INFO: pip is looking at multiple versions of transformers[torch] to determine which version is compatible with other requirements. This could take a while.
  Obtaining dependency information for transformers[torch] from https://files.pythonhosted.org/packages/0f/12/d8e27a190ca67811f81deea3183b528d9169f10b74d827e0b9211520ecfa/transformers-4.36.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for transformers[torch] from https://files.pythonhosted.org/packages/92/ba/cfff7e01f7070d9fca3964bf42b2257b86964c3e6763b8d5435436cc1d77/transformers-4.35.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 5.8 MB/s eta 0:00:00
  Obtaining dependency in

In [4]:
pip show accelerate

Name: accelerate
Version: 0.20.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /opt/conda/lib/python3.10/site-packages
Requires: numpy, packaging, psutil, pyyaml, torch
Required-by: catalyst
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install evaluate

In [ ]:
pip install --upgrade datasets

In [66]:
import os
import pandas as pd
import numpy as np
from datasets import Dataset,DatasetDict
from transformers import DataCollatorWithPadding
import evaluate
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import AutoTokenizer
from transformers import Trainer, TrainingArguments
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [67]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

In [68]:
data = pd.read_csv("/kaggle/input/dataset-dina/train.csv")

In [69]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [70]:
data.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [71]:
df_toxic = data.copy()
df_toxic['is_toxic'] = df_toxic[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].apply(lambda row: any(row), axis=1).astype(int)
df_toxic = df_toxic.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1)

In [72]:
df_toxic

,id,comment_text,is_toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0


In [73]:
df_toxic = df_toxic.drop('id', axis=1)

In [74]:
df_toxic.head(5)

,comment_text,is_toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [75]:
df_toxic['is_toxic'].value_counts()

is_toxic
0    143346
1     16225
Name: count, dtype: int64

In [76]:
df_toxic_balanced = pd.concat([
    df_toxic[df_toxic['is_toxic'] == 0].sample(2000, random_state=42),
    df_toxic[df_toxic['is_toxic'] == 1].sample(2000, random_state=42)
])

df_toxic_balanced = df_toxic_balanced.sample(frac=1, random_state=42)

print(df_toxic_balanced['is_toxic'].value_counts())

is_toxic
0    2000
1    2000
Name: count, dtype: int64


In [77]:
df_toxic_balanced.dtypes

comment_text    object
is_toxic         int64
dtype: object

In [78]:
df_toxic_balanced = df_toxic_balanced.rename(columns={'is_toxic': 'label'})
df_toxic_balanced['label'] = df_toxic_balanced['label'].astype(float)

In [79]:
df_toxic_balanced.dtypes

comment_text     object
label           float64
dtype: object

In [80]:
ds = Dataset.from_pandas(df_toxic_balanced)

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [81]:
ds

Dataset({
    features: ['comment_text', 'label', '__index_level_0__'],
    num_rows: 4000
})

In [82]:
pretrained_model = 'distilbert-base-cased'

In [84]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model, use_fast =True) # False

In [109]:
def preprocess(x):
    return tokenizer(x["comment_text"], truncation=True, max_length=128)

In [110]:
tok_ds = ds.map(preprocess, batched=True)
tok_ds

  0%|          | 0/4 [00:00<?, ?ba/s]

Dataset({
    features: ['comment_text', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 4000
})

In [111]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='max_length')

In [112]:
dataset = tok_ds.train_test_split(test_size=0.3)

In [113]:
dataset

DatasetDict({
    train: Dataset({
        features: ['comment_text', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 2800
    })
    test: Dataset({
        features: ['comment_text', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 1200
    })
})

In [114]:
tok_train_dataset = dataset["train"]
tok_test_dataset = dataset["test"]

In [115]:
accuracy = evaluate.load("accuracy")

In [116]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [117]:
def compute_metrics(pred):
    labels = pred.label_ids.float()
    preds = pred.predictions.argmax(-1).float()
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [118]:
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model, num_labels=1, output_attentions=True)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight

In [119]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    fp16=True,
    # gradient_checkpointing=True, #according to Bing Chat this will do a trade-off between speed and memory usage, but distilbert doesn't support it
    #per_device_train_batch_size=2,#16 is better, but requires more RAM
    #per_device_eval_batch_size=2, #16 is better, but requires more RAM
    num_train_epochs=2,
    weight_decay=0.01,
    #evaluation_strategy="epoch",
    #save_strategy="epoch",
    #load_best_model_at_end=True
)

In [120]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tok_train_dataset,
    eval_dataset=tok_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


TrainOutput(global_step=350, training_loss=0.08759590148925782, metrics={'train_runtime': 95.0849, 'train_samples_per_second': 58.895, 'train_steps_per_second': 3.681, 'total_flos': 741804203212800.0, 'train_loss': 0.08759590148925782, 'epoch': 2.0})

In [121]:
trainer.save_model("comments_model_v2")

In [122]:
model_save_path = '/kaggle/working/comments_model_v2.pkl'

with open(model_save_path, 'wb') as model_file:
    pickle.dump(model, model_file)

In [123]:
model_pickle_path = '/kaggle/working/comments_model_v2.pkl'

with open(model_pickle_path, 'rb') as model_file:
    model = pickle.load(model_file)

In [125]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline
 
model_path = "/kaggle/working/comments_model_v2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
 
pipeline =  TextClassificationPipeline(model=model, tokenizer=tokenizer)
print(pipeline("D'aww! He matches this background colour I'm seemingly stuck with. Thanks."))

[{'label': 'LABEL_0', 'score': 0.5156227946281433}]


In [52]:
import json

config_path = "/kaggle/working/comments_model/config.json"

with open(config_path, 'r') as config_file:
    config = json.load(config_file)

config


{'_name_or_path': 'distilbert-base-cased',
 'activation': 'gelu',
 'architectures': ['DistilBertForSequenceClassification'],
 'attention_dropout': 0.1,
 'dim': 768,
 'dropout': 0.1,
 'hidden_dim': 3072,
 'id2label': {'0': 'LABEL_0'},
 'initializer_range': 0.02,
 'label2id': {'LABEL_0': 0},
 'max_position_embeddings': 512,
 'model_type': 'distilbert',
 'n_heads': 12,
 'n_layers': 6,
 'output_attentions': True,
 'output_past': True,
 'pad_token_id': 0,
 'problem_type': 'regression',
 'qa_dropout': 0.1,
 'seq_classif_dropout': 0.2,
 'sinusoidal_pos_embds': False,
 'tie_weights_': True,
 'torch_dtype': 'float32',
 'transformers_version': '4.29.2',
 'vocab_size': 28996}